In [4]:
# vocabulary: abc def -
#             123 456 0
# lexicon: cab bed
# sample n=6 m=2
# bd- dee
# dee ffa
# answer: bed

In [5]:
# CTC matrix
import torch

lexicon = [[3, 1, 2], [2, 5, 4]]
indices = torch.tensor([[2, 4], [4, 5], [0, 5], [4, 6], [5, 6], [5, 1]])
indices = torch.tensor([[2, 4, 0, 4, 5, 5], [4, 5, 5, 6, 6, 1]])

In [24]:
import scipy.sparse as sp
import numpy as np

# build default grid n=6 m=2
adjacencies = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5],  # 1st row
               [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], # 2nd row
               [0, 7], [1, 8], [2, 9], [3, 10], [4, 11], # \\\
               [6, 1], [7, 2], [8, 3], [9, 4], [10, 5]] # ///

# change edges for every -
adjacencies = [[0, 1], [1, 3], [3, 4], [4, 5],
               [6, 7], [7, 8], [8, 9], [7, 9], [9, 10], [10, 11],
               [0, 7], [1, 8], [1, 9], [3, 10], [4, 11],
               [6, 1], [7, 3], [8, 3], [9, 4], [10, 5]]


In [29]:
def generate_adj_sp(edges, n_entities):
    '''
    Build adjacency matrix
    '''
    adj_shape = (n_entities, n_entities)
    
    # split subject (row) and object (col) node URIs
    n_edges = len(edges)
    row, col = np.transpose(edges)


    # create adjacency matrix for this predicate
    data = np.ones(n_edges)
    adj = sp.csr_matrix((data, (row, col)), shape=adj_shape)
    return adj

A = generate_adj_sp(adjacencies, n_entities=12)
print(A.shape)

(12, 12)


In [40]:
# activate nodes that match lexicon words: b22 e53
# be 23 -> ed 23
_map = {'e': [4, 5, 7, 8], 'd': [1, 3, 6]}
indices =    [2, 4, 0, 4, 5, 5, 4, 5, 5, 6, 6, 1]
x = np.array([2, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0])
print(x.shape)

(12,)


In [41]:
# MP
y = x @ A
print(y)

[0. 2. 0. 6. 0. 3. 0. 2. 3. 6. 0. 3.]


In [96]:
# check patterns in y using indices e5d4 23
e_activations = y[_map['e']]
print(e_activations)
e_activations = e_activations[e_activations!=0]
e_activations = e_activations[e_activations%2==0]
print(e_activations)  # 3rd e 7

d_activations = y[_map['d']]
print(d_activations)  # 2nd d 3
d_activations = d_activations[d_activations!=0]
d_activations = d_activations[d_activations%3==0]
print(d_activations)


print(bool(len(e_activations) & len(d_activations)))

[7, 3] in adjacencies

[0. 3. 2. 3.]
[2.]
[2. 6. 0.]
[6.]
True


True